In [1]:
from orion import Orion
import pandas as pd
import shutil
import os
import csv

In [ ]:
import glob
import os.path

dataset = "MSL"
experiment = "test_4"

folder_path = r'path where your files are located'
file_type = r'\*type'
files = glob.glob(folder_path + file_type)
max_file = max(files, key=os.path.getctime)

print(max_file)

In [2]:
#specify which dataset you want to train on
dataset = "MSL"

In [3]:
#set path where all folders to run code are
base_path = "/home/lunet/cosoc/Desktop/orion_anomaly_pipeline"


In [4]:
#set number of iterations
iterations = 4

In [5]:
#set experiment name
experiment = "test_4"

In [6]:
#set the hyperparameters for training
hyperparameters = {
    "mlprimitives.custom.timeseries_preprocessing.time_segments_aggregate#1": {
        "time_column": "timestamp",
        "interval": 1,
        "method": "mean"
    },
    "orion.primitives.tadgan.TadGAN#1": {
        "epochs": 3,
        "batch_size": 64
    },
    "orion.primitives.tadgan.score_anomalies#1": {
        "rec_error_type": "dtw",
        "comb": "mult"
    }
}

In [7]:
#gets sensor names for dataset specified
def get_dataset_sensors():
    
    if dataset == "MSL":
        sensors = ["M-6", "M-1", "M-2", "S-2", "P-10", "T-4", "T-5", "F-7", "M-3", "M-4", "M-5", "P-15", "C-1", "C-2", "T-12", "T-13", "F-4", "F-5", "D-14", "T-9", "P-14", "T-8", "P-11", "D-15", "D-16", "M-7", "F-8"]
        return sensors 
    
    elif dataset == "SMAP":
        sensors = ["P-1", "S-1", "E-1", "E-2", "E-3", "E-4", "E-5", "E-6", "E-7", "E-8", "E-9", "E-10", "E-11", "E-12", "E-13", "A-1", "D-1", "P-2", "P-3", "D-2", "D-3", "D-4", "A-2", "A-3", "A-4", "G-1", "G-2", "D-5", "D-6", "D-7", "F-1", "P-4", "G-3", "T-1", "T-2", "D-8", "D-9", "F-2", "G-4", "T-3", "D-11", "D-12", "B-1", "G-6", "G-7", "P-7", "R-1", "A-5", "A-6", "A-7", "D-13", "P-2", "A-8", "A-9", "F-3"]
        return sensors 
    else:
        raise Exception("Dataset name not recognised")

In [8]:
#gets the training, test or labels based on mode
def get_data(sensor, mode):
    
    path = os.path.join(base_path, "datasets", dataset, mode, sensor) + ".csv"
    data = pd.read_csv(path)
    
    return data

In [9]:
#grabs the directory of the model and creates a directory if it doesnt exist
def get_model_dir(sensor, model_name):
    
    path = os.path.join(base_path, "models", experiment, dataset, sensor)
    os.makedirs(path, exist_ok = True)
    model_path = os.path.join(path, model_name)
    
    return model_path

In [10]:
#saves the trained model
def save_model(model_path, model):
    Orion.save(model, path=model_path)

In [11]:
#saves the training by taking the temp log file outputted from the library hard coded in tadgan.py
def save_training_log(model_name, sensor):
    path = os.path.join(base_path, "training_logs")
    old_path = os.path.join(path, "temp/loss.csv")
    new_path = os.path.join(path, experiment, dataset, sensor)
    os.makedirs(new_path, exist_ok = True)
    new_name = os.path.join(new_path, model_name) + ".csv"
    os.replace(old_path, new_name)

In [12]:
#evaluates the model on the test set and saves results
def evaluate_model(model, model_name, test_data, ground_truth):
    import orion.evaluation
    
    detected_anomalies = model.detect(test_data)
    
    results = {'model_name': model_name}
    
    
    confus_matrix = orion.evaluation.contextual_confusion_matrix(expected=ground_truth, observed=detected_anomalies, data=test_data, weighted=False)
    results['ovrlap_T_N'] = confus_matrix[0]
    results['ovrlap_F_P'] = confus_matrix[1]
    results['ovrlap_F_N'] = confus_matrix[2]
    results['ovrlap_T_P'] = confus_matrix[3]
    results['ovrlap_precision'] = orion.evaluation.contextual_precision(expected=ground_truth, observed=detected_anomalies, data=test_data, weighted=False)
    results['ovrlap_recall'] = orion.evaluation.contextual_recall(expected=ground_truth, observed=detected_anomalies, data=test_data, weighted=False)
    results['ovrlap_f1'] = orion.evaluation.contextual_f1_score(expected=ground_truth, observed=detected_anomalies, data=test_data, weighted=False)

    for i in range(len(ground_truth.index)):
        start = ground_truth['start'][i]
        end = ground_truth['end'][i]
        results["fault_" + str(i) + "_acc_weighted"] = orion.evaluation.contextual_accuracy(expected=ground_truth, observed=detected_anomalies, start=start, end=end, weighted=True)
        results["fault_" + str(i) + "_prec_weighted"] = orion.evaluation.contextual_precision(expected=ground_truth, observed=detected_anomalies, start=start, end=end, weighted=True)
        results["fault_" + str(i) + "_recall_weighted"] = orion.evaluation.contextual_recall(expected=ground_truth, observed=detected_anomalies, start=start, end=end, weighted=True)
        results["fault_" + str(i) + "_f1_weighted"] = orion.evaluation.contextual_f1_score(expected=ground_truth, observed=detected_anomalies, start=start, end=end, weighted=True)    
    
    return results

In [13]:
#saves results to either new results file or appends to existing one
def save_results(results, sensor):
    path = os.path.join(base_path, "results", experiment, dataset, sensor)
    os.makedirs(path, exist_ok = True)
    results_path = path + ".csv"
    
    if not os.path.exists(results_path):
        with open(file=results_path, mode="a") as file:
            writer_object = csv.writer(file)
            writer_object.writerow(list(results.keys()))
            file.close()

    with open(file=results_path, mode="a") as file:
        writer_object = csv.writer(file)
        writer_object.writerow(results.values())
        file.close()

In [14]:
sensors = get_dataset_sensors()

for sensor in sensors:
    
    train_data = get_data(sensor, mode="train")
    test_data = get_data(sensor, mode="test")
    ground_truth = get_data(sensor, mode="labels")
    
    print("Begun training sensor: " + dataset + "-" + sensor + " | sensor: " + str(sensors.index(sensor) + 1) + "/" + str(len(sensors)))
    
    for i in range(iterations):
        orion = Orion(pipeline='tadgan.json',hyperparameters=hyperparameters)
        print("Begun training " + str(i + 1) + "/" + str(iterations) + " iterations for: " + dataset + "-" + sensor + " | sensor: " + str(sensors.index(sensor) + 1) + "/" + str(len(sensors)))
        orion.fit(train_data)
        model_name = sensor + "_" + str(i)
        model_path = get_model_dir(sensor=sensor, model_name=model_name)
        save_model(model_path=model_path, model=orion)
        save_training_log(model_name=model_name, sensor=sensor)
        results = evaluate_model(model=orion, model_name=model_name, test_data=test_data, ground_truth=ground_truth)
        save_results(results=results, sensor=sensor)

Begun training sensor: MSL-M-6 | sensor: 1/27


2023-05-02 17:51:36.280219: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


Begun training 1/4 iterations for: MSL-M-6 | sensor: 1/27


2023-05-02 17:51:46.723262: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2023-05-02 17:51:46.809074: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-02 17:51:46.810124: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.545GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2023-05-02 17:51:46.810205: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2023-05-02 17:51:47.485617: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2023-05-02 17:51:47.724055: I tensorflow/stream_executor

Epoch: 1/3, Losses: {'cx_loss': 1.4082, 'cz_loss': 2.4731, 'eg_loss': 8.0512}
Epoch: 2/3, Losses: {'cx_loss': -9.0353, 'cz_loss': 4.7451, 'eg_loss': -2.9666}
Epoch: 3/3, Losses: {'cx_loss': -10.3842, 'cz_loss': 7.7196, 'eg_loss': -9.2092}
Begun training 2/4 iterations for: MSL-M-6 | sensor: 1/27
Epoch: 1/3, Losses: {'cx_loss': 0.7208, 'cz_loss': 2.5167, 'eg_loss': 7.8509}
Epoch: 2/3, Losses: {'cx_loss': -9.1725, 'cz_loss': 4.6834, 'eg_loss': -2.0022}
Epoch: 3/3, Losses: {'cx_loss': -10.4414, 'cz_loss': 4.7831, 'eg_loss': -7.2118}
Begun training 3/4 iterations for: MSL-M-6 | sensor: 1/27
Epoch: 1/3, Losses: {'cx_loss': -0.8303, 'cz_loss': 2.9461, 'eg_loss': 6.6232}
Epoch: 2/3, Losses: {'cx_loss': -9.5927, 'cz_loss': 8.4363, 'eg_loss': -2.3618}
Epoch: 3/3, Losses: {'cx_loss': -9.7366, 'cz_loss': 9.3628, 'eg_loss': -6.6372}
Begun training 4/4 iterations for: MSL-M-6 | sensor: 1/27
Epoch: 1/3, Losses: {'cx_loss': -0.6689, 'cz_loss': 3.0007, 'eg_loss': 7.0551}
Epoch: 2/3, Losses: {'cx_loss'

...

Invalid value encountered for precision 0.0/ recall 0.0.
Traceback (most recent call last):
  File "/home/lunet/cosoc/.conda/envs/orion_test_1/lib/python3.7/site-packages/orion/evaluation/common.py", line 70, in _f1_score
    return 2 * (precision * recall) / (precision + recall)
ZeroDivisionError: float division by zero
/home/lunet/cosoc/.conda/envs/orion_test_1/lib/python3.7/site-packages/orion/evaluation/common.py:70: RuntimeWarning: invalid value encountered in double_scalars
  return 2 * (precision * recall) / (precision + recall)


Begun training 3/4 iterations for: MSL-M-1 | sensor: 2/27
Epoch: 1/3, Losses: {'cx_loss': 2.2203, 'cz_loss': 2.6519, 'eg_loss': 1.1888}
Epoch: 2/3, Losses: {'cx_loss': -1.6934, 'cz_loss': 5.5706, 'eg_loss': -2.3845}
Epoch: 3/3, Losses: {'cx_loss': -1.3037, 'cz_loss': 10.6369, 'eg_loss': -5.9015}


Invalid value encountered for precision 0.0/ recall 0.0.
Traceback (most recent call last):
  File "/home/lunet/cosoc/.conda/envs/orion_test_1/lib/python3.7/site-packages/orion/evaluation/common.py", line 70, in _f1_score
    return 2 * (precision * recall) / (precision + recall)
ZeroDivisionError: float division by zero
/home/lunet/cosoc/.conda/envs/orion_test_1/lib/python3.7/site-packages/orion/evaluation/common.py:70: RuntimeWarning: invalid value encountered in double_scalars
  return 2 * (precision * recall) / (precision + recall)


Begun training 4/4 iterations for: MSL-M-1 | sensor: 2/27
Epoch: 1/3, Losses: {'cx_loss': 1.6794, 'cz_loss': 1.8325, 'eg_loss': 0.9393}
Epoch: 2/3, Losses: {'cx_loss': -1.0982, 'cz_loss': 5.0487, 'eg_loss': -2.9952}
Epoch: 3/3, Losses: {'cx_loss': -0.9203, 'cz_loss': 7.6253, 'eg_loss': -4.5306}
Begun training sensor: MSL-M-2 | sensor: 3/27
Begun training 1/4 iterations for: MSL-M-2 | sensor: 3/27
Epoch: 1/3, Losses: {'cx_loss': 0.0689, 'cz_loss': 3.0997, 'eg_loss': 1.6958}
Epoch: 2/3, Losses: {'cx_loss': -5.5166, 'cz_loss': 9.1205, 'eg_loss': -9.3866}
Epoch: 3/3, Losses: {'cx_loss': -3.0647, 'cz_loss': 18.1149, 'eg_loss': -18.7862}
Begun training 2/4 iterations for: MSL-M-2 | sensor: 3/27
Epoch: 1/3, Losses: {'cx_loss': -1.655, 'cz_loss': 3.6539, 'eg_loss': 1.6466}
Epoch: 2/3, Losses: {'cx_loss': -6.0614, 'cz_loss': 7.5672, 'eg_loss': -8.1528}
Epoch: 3/3, Losses: {'cx_loss': -4.5724, 'cz_loss': 5.6619, 'eg_loss': -11.383}
Begun training 3/4 iterations for: MSL-M-2 | sensor: 3/27
Epoch:

KeyboardInterrupt: 

In [ ]:
from orion.data import load_signal

In [ ]:
df = load_signal('S-1-train')

In [ ]:
sensors = ["a", "b", "c"]